In [25]:
# GOAL:
# Do a simplified version of the data-cleaning necessary for my most recent project.
# Create data with one observation *per college per day*
# incorporating data from:
# IPEDS 2018: IPEDS/cdsfile_all_STATA_RV_942020-662.dta -> get percent in distance ed, and tuition share (f1tufeft/(f1stapft+f1lcapft+f1gvgcft+f1pggcft+f1invrft+f1otrvft+f1endmft))
# IPEDS 2019: IPEDS/cdsfile_all_STATA_RV_942020-417.dta -> get whether the college is private
# EADA: EADA/InstLevel.xlsx -> get whether hte college is Division I in sports (ClassificationCode is 1 through 3)
# NY Times: NYT/us-counties_cases.csv -> Get by-county information on COVID cases on July 31
# Census: Census/co-est2019-annres.csv -> Get by-county population in 2019
# politicaldata house_116 data -> Get 2018 congressoinal DW-Nominate scores
# foot_traffic_panel.Rdata -> Pre-prepared college-day file of foot traffic visits to nearby locations

import pandas as pd
import numpy as np

# IPEDS 2018
ipeds2018 = pd.read_stata("IPEDS/cdsfile_all_STATA_RV_942020-662.dta")
ipeds2018['tuition_share'] = (ipeds2018['f1tufeft']/(
    ipeds2018['f1stapft']+ipeds2018['f1lcapft']+ipeds2018['f1gvgcft']+
    ipeds2018['f1pggcft']+ipeds2018['f1invrft']+ipeds2018['f1otrvft']+ipeds2018['f1endmft']))

keep = ['unitid', 'pctdeexc', 'tuition_share']

ipeds2018 = ipeds2018[keep]

ipeds2018



,unitid,pctdeexc,tuition_share
0,100654,2.0,NaN
1,100663,26.0,0.160065
2,100706,7.0,0.286900
3,100724,10.0,0.245470
4,100751,10.0,0.362953
...,...,...,...
3731,494597,NaN,NaN
3732,494603,NaN,NaN
3733,494630,NaN,NaN
3734,494685,NaN,NaN


In [46]:
# IPEDS 2019
ipeds2019 = pd.read_stata("IPEDS/cdsfile_all_STATA_RV_942020-417.dta")

ipeds2019['Private'] = ipeds2019['sector'].apply(lambda x: x in ['Private not-for-profit, 4-year or above',
                                              'Private not-for-profit, 2-year',
                                              'Private not-for-profit, less-than 2-year'])

ipeds2019 = ipeds2019[['unitid','Private']]

ipeds_linker = pd.read_stata("IPEDS/cdsfile_all_STATA_RV_942020-417.dta", convert_categoricals = False)
ipeds_linker = ipeds_linker[['unitid', 'countycd']].rename({'countycd':'fips'}, axis = 1)

ipeds_linker

,unitid,fips
0,100654,1089
1,100663,1073
2,100706,1089
3,100724,1101
4,100751,1125
...,...,...
3731,494597,12057
3732,494603,48439
3733,494630,48029
3734,494685,29183


In [20]:
eada = pd.read_csv('EADA/InstLevel.csv')

eada['DivisionOne'] = eada['ClassificationCode'] < 4

eada = eada[['unitid','DivisionOne']]

eada

,unitid,DivisionOne
0,100654,True
1,100663,True
2,100706,False
3,100724,True
4,100751,True
...,...,...
2069,489201,False
2070,489937,False
2071,490805,False
2072,492069,False


In [79]:
nyt = pd.read_csv('NYT/us-counties_cases.csv')

nyt = nyt.loc[(nyt['date'] == '2020-07-31')]

nyt['County'] = nyt['county'] + ' County, ' + nyt['state']

nyt = nyt[['County','fips','cases']]
nyt = nyt.dropna()

nyt['fips'] = nyt['fips'].astype(int)

nyt

,County,fips,cases
385994,"Autauga County, Alabama",1001,1015
385995,"Baldwin County, Alabama",1003,3101
385996,"Barbour County, Alabama",1005,598
385997,"Bibb County, Alabama",1007,363
385998,"Blount County, Alabama",1009,767
...,...,...,...
389206,"Sweetwater County, Wyoming",56037,240
389207,"Teton County, Wyoming",56039,335
389208,"Uinta County, Wyoming",56041,254
389209,"Washakie County, Wyoming",56043,47


In [75]:
census = pd.read_csv('Census/county_simple.csv')

census['2019'] = census['2019'].str.replace(',','').astype(int)

census = census[['County','2019']].rename({'2019':'Population'}, axis =1 )

census['County'] = census['County'].apply(lambda x: x[1:])

census

,County,Population
0,"Autauga County, Alabama",55869
1,"Baldwin County, Alabama",223234
2,"Barbour County, Alabama",24686
3,"Bibb County, Alabama",22394
4,"Blount County, Alabama",57826
...,...,...
3137,"Sweetwater County, Wyoming",42343
3138,"Teton County, Wyoming",23464
3139,"Uinta County, Wyoming",20226
3140,"Washakie County, Wyoming",7805


In [86]:
fulldata = ipeds2018.merge(ipeds2019, on = 'unitid', how = 'outer')

fulldata = fulldata.merge(eada, on = 'unitid', how = 'left')
fulldata.loc[fulldata['DivisionOne'].apply(lambda x: np.isnan(x)), 'DivisionOne'] = False

fulldata = fulldata.merge(ipeds_linker, on = 'unitid', how = 'outer')

fulldata = fulldata.merge(nyt, on = 'fips', how = 'left')

fulldata = fulldata.merge(census, on = 'County', how = 'right')
fulldata = fulldata.loc[fulldata['unitid'].apply(lambda x: np.isnan(x))]

fulldata[1:20]

,unitid,pctdeexc,tuition_share,Private,DivisionOne,fips,County,cases,Population
3,NaN,NaN,NaN,NaN,NaN,NaN,"Barbour County, Alabama",NaN,24686
4,NaN,NaN,NaN,NaN,NaN,NaN,"Bibb County, Alabama",NaN,22394
5,NaN,NaN,NaN,NaN,NaN,NaN,"Blount County, Alabama",NaN,57826
6,NaN,NaN,NaN,NaN,NaN,NaN,"Bullock County, Alabama",NaN,10101
7,NaN,NaN,NaN,NaN,NaN,NaN,"Butler County, Alabama",NaN,19448
9,NaN,NaN,NaN,NaN,NaN,NaN,"Chambers County, Alabama",NaN,33254
10,NaN,NaN,NaN,NaN,NaN,NaN,"Cherokee County, Alabama",NaN,26196
11,NaN,NaN,NaN,NaN,NaN,NaN,"Chilton County, Alabama",NaN,44428
12,NaN,NaN,NaN,NaN,NaN,NaN,"Choctaw County, Alabama",NaN,12589
13,NaN,NaN,NaN,NaN,NaN,NaN,"Clarke County, Alabama",NaN,23622


In [27]:
np.isnan(np.nan)

True

In [76]:
nyt = pd.read_csv('NYT/us-counties_cases.csv')

nyt

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
498891,2020-09-03,Sweetwater,Wyoming,56037.0,304,2
498892,2020-09-03,Teton,Wyoming,56039.0,435,1
498893,2020-09-03,Uinta,Wyoming,56041.0,305,2
498894,2020-09-03,Washakie,Wyoming,56043.0,108,6
